In [1]:
import os
import getpass

if not os.getenv("GEMINI_API_KEY"):
    os.environ["GEMINI_API_KEY"] = getpass.getpass("Masukkan GEMINI_API_KEY: ")

os.environ["GOOGLE_API_KEY"] = os.environ["GEMINI_API_KEY"]
print("Key OK")


Masukkan GEMINI_API_KEY: ··········
Key OK


##**Debug Helper (Traceback)**

In [2]:
import traceback, sys
def print_exc():
    exc_type, exc_value, exc_tb = sys.exc_info()
    traceback.print_exception(exc_type, exc_value, exc_tb)

##**Install & Setup Environment (CrewAI + LiteLLM + Google GenAI)l**

In [3]:
!pip uninstall -y \
  langchain langchain-core langchain-community langchain-google-genai \
  google-generativeai google-genai \
  crewai crewai-tools litellm -q

!pip install --upgrade pip -q

!pip install --no-cache-dir -q \
  crewai \
  crewai-tools \
  litellm \
  google-genai \
  pandas numpy scikit-learn

print("Install bersih sukses: CrewAI + LiteLLM + google-genai (tanpa langchain)")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 33.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langgraph 1.0.4 requires langchain-core>=0.1, which is not installed.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
opentelemetry-exporter-gcp-logging 1.11.0a0 requires opentelemetry-api>=1.35.0, but you have opentelemetry-api 1.34.1 which is incompatible.
opentelemetry-exporter-gcp-logging 1.11.0a0 requires opentelemetry-sdk<1.39.0,>=1.35.0, but you have opentelemetry-sdk 1.34.1 which is incompatible.
transformers 4.57.3 

##**Import Library Dasar**

In [4]:
import os, ast
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

from crewai import Agent, Task, Crew, Process, LLM
from crewai.tools import tool
from google import genai

print("Import dasar OK")

Import dasar OK


##**Konfigurasi API Key + Embedding Function**

In [5]:
# Cell 3 — Setup API key + embedding client

import os
from google import genai

REAL_KEY = os.environ.get("GEMINI_API_KEY")
if not REAL_KEY:
    raise ValueError("API key belum di-set. Isi GEMINI_API_KEY dulu di environment.")

# set env biar dipakai semua cell bawahnya
os.environ["GEMINI_API_KEY"] = REAL_KEY
os.environ["GOOGLE_API_KEY"] = REAL_KEY

client = genai.Client(api_key=REAL_KEY)
EMBED_MODEL = "text-embedding-004"

def get_embedding(text: str):
    try:
        res = client.models.embed_content(
            model=EMBED_MODEL,
            contents=text,
        )
        return res.embeddings[0].values
    except Exception as e:
        print("Error embedding:", e)
        return {"error": str(e)}

print("API & embedding siap")

API & embedding siap


In [6]:
# Cell A — Helper LLM Gemini 2.5 Flash (Mode Hemat)

GEN_MODEL = "gemini-2.5-flash"  # model yang kamu pakai di capstone

def call_gemini_once(prompt: str) -> str:
    """
    Panggil Gemini 2.5 Flash SATU KALI dengan prompt teks.
    Dipakai di mode hemat supaya tidak banyak request ke LLM.
    """
    try:
        res = client.models.generate_content(
            model=GEN_MODEL,
            contents=[prompt],
        )
        # library google-genai biasanya punya property .text
        try:
            return res.text
        except AttributeError:
            # fallback kalau format beda
            return str(res)
    except Exception as e:
        print("Error LLM (Gemini):", e)
        return f"[ERROR LLM] {e}"

##**Load Dataset CSV + Parse Embedding**

In [7]:
# Cell 5 — Load Dataset (versi tahan CSV rusak)
import pandas as pd
import ast

print("Memuat data yang sudah benar...")

chemicals = pd.read_csv("/content/chemicals_with_embeddings (3).csv")
products  = pd.read_csv("/content/products_with_embeddings (3).csv")

# relations sering error karena ada 1 baris kutipnya kebuka
relations = pd.read_csv(
    "/content/relations_with_embeddings (2).csv",
    engine="python",      # parser lebih toleran
    on_bad_lines="skip"   # lewatin baris yang rusak
)

print("File kebaca semua. Mulai parsing embedding...")

def safe_literal_eval(x):
    """Biar kalau ada embedding yang rusak ga bikin crash."""
    if not isinstance(x, str):
        return x
    try:
        return ast.literal_eval(x)
    except:
        return None  # tandai rusak jadi None

# Parse kolom embedding biar jadi list angka
for name, df in [("chemicals", chemicals), ("products", products), ("relations", relations)]:
    if "embedding" in df.columns and len(df) > 0:
        if isinstance(df["embedding"].iloc[0], str):
            df["embedding"] = df["embedding"].apply(safe_literal_eval)

        # buang baris yang embedding-nya gagal diparse
        before = len(df)
        df.dropna(subset=["embedding"], inplace=True)
        after = len(df)

        print(f" Data {name} siap! (Jumlah baris: {after}, buang {before-after} baris embedding rusak)")
    else:
        print(f" Data {name} siap! (Jumlah baris: {len(df)})")

print("\nSIAP! Langsung jalankan cell berikutnya (Tools, Agent, dll).")


Memuat data yang sudah benar...
File kebaca semua. Mulai parsing embedding...
 Data chemicals siap! (Jumlah baris: 2000, buang 0 baris embedding rusak)
 Data products siap! (Jumlah baris: 1500, buang 0 baris embedding rusak)
 Data relations siap! (Jumlah baris: 2551, buang 0 baris embedding rusak)

SIAP! Langsung jalankan cell berikutnya (Tools, Agent, dll).


In [8]:
# Cell 4.2 — Mapping ID -> Nama (buat output)

assert "compound_id" in chemicals.columns
assert "compound_name" in chemicals.columns
assert "product_id" in products.columns
assert "product_name" in products.columns

compound_id_to_name = dict(zip(
    chemicals["compound_id"].astype(str),
    chemicals["compound_name"].astype(str)
))

product_id_to_name = dict(zip(
    products["product_id"].astype(str),
    products["product_name"].astype(str)
))

def pretty_compound(cid: str) -> str:
    cid = str(cid)
    nm = compound_id_to_name.get(cid)
    return f"{cid} ({nm})" if nm and nm != "nan" else cid

def pretty_product(pid: str) -> str:
    pid = str(pid)
    nm = product_id_to_name.get(pid)
    return f"{pid} ({nm})" if nm and nm != "nan" else pid

print("Mapping siap")

Mapping siap


##**Semantic Search Function**

In [9]:
def semantic_search(df, query, top_k=5):
    q_emb_result = get_embedding(query)
    if isinstance(q_emb_result, dict) and "error" in q_emb_result:
        return {"error": q_emb_result["error"]}
    q_emb = q_emb_result

    if "embedding" not in df.columns or df["embedding"].isnull().all():
        return {"error": "DataFrame tidak punya embedding valid."}

    df_temp = df.copy()
    df_temp["score"] = df_temp["embedding"].apply(
        lambda x: cosine_similarity([q_emb], [x])[0][0]
    )

    if "compound_name" in df_temp.columns:
        df_temp["name_for_output"] = df_temp["compound_name"]
        info_cols = [
            "molecular_formula","molecular_weight","reactivity_score",
            "toxicity_level","solubility","function","origin",
            "stability_index","skin_absorption_rate","ph_value"
        ]
        df_temp["info_for_output"] = df_temp[info_cols].fillna("").astype(str).agg(", ".join, axis=1)

    elif "product_name" in df_temp.columns:
        df_temp["name_for_output"] = df_temp["product_name"]
        info_cols = [
            "category","target_skin_type","key_ingredients",
            "effectiveness_score","safety_score","popularity_index","description"
        ]
        df_temp["info_for_output"] = df_temp[info_cols].fillna("").astype(str).agg(", ".join, axis=1)

    elif "relation_id" in df_temp.columns:
        df_temp["name_for_output"] = df_temp["relation_id"]
        info_cols = [
            "product_id","compound_id","percentage_in_formula",
            "role_in_product","synergy_score","interaction_type",
            "potential_new_compound"
        ]
        df_temp["info_for_output"] = df_temp[info_cols].fillna("").astype(str).agg(", ".join, axis=1)

    else:
        return {"error": "Kolom name/info tidak ketemu."}

    return (
        df_temp.sort_values("score", ascending=False)
        .head(top_k)[["name_for_output","info_for_output","score"]]
        .rename(columns={"name_for_output":"name","info_for_output":"info"})
    )

print("Semantic search siap")

Semantic search siap


##**Tools untuk Agent (BaseTool via @tool)**

In [10]:
# Cell 6 — semantic_search (output ID untuk mapping nama)

def semantic_search(df, query, top_k=5):
    q_emb_result = get_embedding(query)
    if isinstance(q_emb_result, dict) and "error" in q_emb_result:
        return {"error": q_emb_result["error"]}
    q_emb = q_emb_result

    if "embedding" not in df.columns or df["embedding"].isnull().all():
        return {"error": "DataFrame tidak punya embedding valid."}

    df_temp = df.copy()
    df_temp["score"] = df_temp["embedding"].apply(
        lambda x: cosine_similarity([q_emb], [x])[0][0]
    )

    # Case 1: chemicals
    if "compound_name" in df_temp.columns:
        df_temp["name_for_output"] = df_temp["compound_id"]
        info_cols = [
            "compound_name",
            "molecular_formula", "molecular_weight", "reactivity_score",
            "toxicity_level", "solubility", "function", "origin",
            "stability_index", "skin_absorption_rate", "ph_value"
        ]
        df_temp["info_for_output"] = (
            df_temp[info_cols]
            .fillna("")
            .astype(str)
            .agg(", ".join, axis=1)
        )

    # Case 2: products
    elif "product_name" in df_temp.columns:
        df_temp["name_for_output"] = df_temp["product_id"]
        info_cols = [
            "product_name",
            "category", "target_skin_type", "key_ingredients",
            "effectiveness_score", "safety_score", "popularity_index",
            "description"
        ]
        df_temp["info_for_output"] = (
            df_temp[info_cols]
            .fillna("")
            .astype(str)
            .agg(", ".join, axis=1)
        )

    # Case 3: relations
    elif "relation_id" in df_temp.columns:
        df_temp["name_for_output"] = df_temp["relation_id"]
        info_cols = [
            "product_id", "compound_id", "percentage_in_formula",
            "role_in_product", "synergy_score", "interaction_type",
            "potential_new_compound"
        ]
        df_temp["info_for_output"] = (
            df_temp[info_cols]
            .fillna("")
            .astype(str)
            .agg(", ".join, axis=1)
        )

    else:
        return {"error": "Kolom name/info tidak ketemu."}

    return (
        df_temp.sort_values("score", ascending=False)
        .head(top_k)[["name_for_output", "info_for_output", "score"]]
        .rename(columns={"name_for_output": "name", "info_for_output": "info"})
    )

print("Semantic search siap.")


# Tools

from crewai.tools import tool

@tool("search_chemical_tool")
def search_chemical_tool(query: str, top_k: int = 5) -> str:
    """Cari & ranking bahan kimia relevan."""
    result = semantic_search(chemicals, query, top_k)
    if isinstance(result, dict) and "error" in result:
        return result["error"]

    result["name_display"] = result["name"].apply(pretty_compound)
    return result.to_json(orient="records")


@tool("search_product_tool")
def search_product_tool(query: str, top_k: int = 5) -> str:
    """Cari & ranking produk relevan."""
    result = semantic_search(products, query, top_k)
    if isinstance(result, dict) and "error" in result:
        return result["error"]

    result["name_display"] = result["name"].apply(pretty_product)
    return result.to_json(orient="records")


@tool("check_innovation_tool")
def check_innovation_tool(compound_id1: str, compound_id2: str) -> str:
    """Cek interaksi/sinergi/inovasi antar compound dari relations."""
    df_filtered = relations[
        ((relations['compound_id'] == compound_id1) & relations['potential_new_compound'].astype(str).str.contains(compound_id2, na=False)) |
        ((relations['compound_id'] == compound_id2) & relations['potential_new_compound'].astype(str).str.contains(compound_id1, na=False)) |
        ((relations['compound_id'] == compound_id1) & (relations['product_id'].isin(relations[relations['compound_id'] == compound_id2]['product_id']))) |
        ((relations['compound_id'] == compound_id2) & (relations['product_id'].isin(relations[relations['compound_id'] == compound_id1]['product_id'])))
    ]

    if df_filtered.empty:
        return f"Tidak ada catatan inovasi/interaksi antara {compound_id1} dan {compound_id2}."
    return df_filtered.to_json(orient="records")


print("Tools valid untuk CrewAI (dengan name_display)")

Semantic search siap.
Tools valid untuk CrewAI (dengan name_display)


##**Setup LLM Gemini untuk CrewAI**

In [11]:
# Cell 7 — LLM Gemini (lebih aman buat free tier)

from crewai import LLM
import os

# ambil key dari env (kamu udah set os.environ["GOOGLE_API_KEY"] di cell API)
GOOGLE_API_KEY = os.environ.get("GOOGLE_API_KEY")

if not GOOGLE_API_KEY:
    llm = None
    print("GOOGLE_API_KEY belum ke-set.")
else:
    llm = LLM(
        model="gemini/gemini-2.5-flash",
        api_key=GOOGLE_API_KEY
    )
    print("LLM Gemini Flash aktif via CrewAI")

LLM Gemini Flash aktif via CrewAI


##**Agent 1: Data Agent**

In [12]:
# Cell 8 — Agent 1: Data Agent

from crewai import Agent

data_agent = Agent(
    role="Data Analyst Agent",
    goal=(
        "Mengambil data bahan/produk relevan, melakukan filtering, dan ranking kandidat terbaik. "
        "Gunakan masing-masing tool maksimal 1 kali dan fokus pada top_k=5."
    ),
    backstory="Ahli database kosmetik yang fokus pada pencarian bahan berdasarkan kebutuhan user.",
    verbose=True,
    allow_delegation=False,
    tools=[search_chemical_tool, search_product_tool],
    llm=llm
)

print("Data Agent siap")

Data Agent siap


##**Agent 2: Chemical Safety Agent**

In [13]:
# Cell 9 — Agent 2: Chemical Safety Agent

chemical_agent = Agent(
    role="Chemical Safety Agent",
    goal="Mengecek kompatibilitas bahan, risiko interaksi, pH ideal, dan potensi iritasi.",
    backstory="Ahli kimia kosmetik dan toksikologi yang sangat hati-hati.",
    verbose=True,
    allow_delegation=False,
    # 🔧 REVISI: sebelumnya hanya [check_innovation_tool]
    tools=[search_chemical_tool, check_innovation_tool],
    llm=llm
)
print("Chemical Agent siap")

Chemical Agent siap


##**Agent 3: Formulation Agent**

In [14]:
# Cell 10 — Agent 3: Formulation Agent

formulation_agent = Agent(
    role="Formulation Agent",
    goal="Membuat formula akhir total 100% + penjelasan + discovery jika ada.",
    backstory="Formulator skincare berpengalaman yang jujur dan hanya menggunakan bahan terverifikasi.",
    verbose=True,
    allow_delegation=False,
    tools=[search_chemical_tool, search_product_tool],
    llm=llm
)

print("Formulation Agent siap")

Formulation Agent siap


##**Define Tasks (Sequential + Expected Output)**

In [15]:
# Cell 11 — Define Tasks (Versi Final: Dataset Bersih)

from crewai import Task

task_data = Task(
    description=(
        "1) DATA AGENT TASK\n"
        "Cari data yang relevan untuk kebutuhan: {topic}.\n"
        "Gunakan tools:\n"
        "- search_product_tool (untuk mencari produk jadi)\n"
        "- search_chemical_tool (untuk mencari bahan aktif spesifik)\n"
        "Outputkan shortlist Produk dan Bahan yang paling relevan dengan masalah user.\n"
        "WAJIB jawab dalam Bahasa Indonesia."
    ),
    expected_output=(
        "Daftar produk dan bahan kimia relevan beserta alasan kenapa dipilih."
    ),
    agent=data_agent
)

task_chemical = Task(
    description=(
        "2) CHEMICAL AGENT TASK\n"
        "Analisis keamanan dan interaksi dari shortlist Data Agent.\n"
        "1. Evaluasi keamanan bahan/produk (pH, iritasi).\n"
        "2. Jika user meminta MENGGABUNGKAN dua hal (Produk+Bahan atau Bahan+Bahan), WAJIB gunakan 'check_innovation_tool' untuk melihat apakah ada reaksi di database.\n"
        "WAJIB jawab dalam Bahasa Indonesia."
    ),
    expected_output=(
        "Analisis keamanan, risiko interaksi, dan hasil pengecekan inovasi (jika ada)."
    ),
    agent=chemical_agent,
    context=[task_data]
)

task_formulation = Task(
    description=(
        "3) FORMULATION AGENT TASK\n"
        "Susun rekomendasi akhir (Formula atau Rutinitas Skincare).\n"
        "ATURAN 'DISCOVERY' (PENEMUAN BARU):\n"
        "- Jika Chemical Agent menemukan 'potential_new_compound', nyatakan itu sebagai 'PENEMUAN SENYAWA BARU' dan jelaskan potensinya.\n"
        "- Jika tidak ada reaksi kimia baru, sebut itu sebagai 'SINERGI FORMULA' (Aman dicampur).\n"
        "\n"
        "ATURAN UMUM:\n"
        "- Gunakan nama bahan/produk asli sesuai data.\n"
        "- Jika formulasi belum lengkap (misal tidak ada Aqua/Pengawet, dsb tapi dibutuhkan oleh best practice & data), tambahkan sebagai '(Rekomendasi Tambahan)' agar aman.\n"
        "WAJIB menjawab dalam Bahasa Indonesia."
    ),
    expected_output=(
        "KELUARKAN HANYA SATU OBJEK JSON VALID (tanpa teks sebelum/sesudah). "
        "Contoh struktur (boleh disesuaikan, tapi tetap JSON):\n"
        "{\n"
        '  "rekomendasi_produk": [ ... ],\n'
        '  "rekomendasi_formula": [ ... ],\n'
        '  "status_inovasi": "penemuan_senyawa_baru" | "sinergi_formula" | "tidak_ada",\n'
        '  "ringkasan_keamanan": "...",\n'
        '  "catatan": "..."\n'
        "}\n"
        "JANGAN menulis penjelasan di luar JSON (tidak ada markdown, tidak ada kalimat pembuka/penutup)."
    ),
    agent=formulation_agent,
    context=[task_data, task_chemical]
)

print("Tasks siap")

Tasks siap


##**Build Crew (Process Sequential)**

In [16]:
# Cell 12 — Build Crew (Sequential) dengan Rate Limiting

from crewai import Crew, Process

crew = Crew(
    agents=[data_agent, chemical_agent, formulation_agent],
    tasks=[task_data, task_chemical, task_formulation],
    process=Process.sequential,
    verbose=False,
    max_rpm=10
)

print("Crew siap (dengan mode santai/anti-error)")

Crew siap (dengan mode santai/anti-error)


##**Safe kickoff**

In [17]:
# Cell 12.5 — Safe kickoff

import time
from google.genai.errors import ClientError

def safe_kickoff(crew, inputs, max_retries=3):
    delay = 4
    last_err = None
    for _ in range(max_retries):
        try:
            return crew.kickoff(inputs=inputs)
        except ClientError as e:
            last_err = e
            if "429" in str(e) or "RESOURCE_EXHAUSTED" in str(e):
                time.sleep(delay)
                delay *= 2
                continue
            raise
    raise RuntimeError(f"Masih kena rate limit. Error terakhir: {last_err}")

In [18]:
# Cell 12.6 — Parser output Agent 3

import json
import ast
import re

def parse_crew_output_to_structured(raw_output: str):
    """
    Parse output Agent 3 ke struktur Python (dict/list) dengan toleransi:
    - ada narasi di luar JSON
    - dibungkus ``` / ```json
    - pakai single quote ala dict Python
    """

    if raw_output is None:
        raise ValueError("Output Agent 3 kosong (None).")

    text = str(raw_output).strip()

    # 1. Buang code fence ```...``` kalau ada
    if text.startswith("```"):
        # hilangkan ```json / ```md / ``` dll di awal
        text = re.sub(r"^```[a-zA-Z0-9_]*", "", text).strip()
        # hilangkan ``` di akhir
        text = re.sub(r"```$", "", text).strip()

    # 2. Ambil blok { ... } paling besar (asumsi JSON utama di situ)
    match = re.search(r"\{.*\}", text, flags=re.DOTALL)
    candidate = match.group(0) if match else text

    # 3. Coba langsung json.loads
    try:
        return json.loads(candidate)
    except Exception:
        pass  # lanjut ke fallback

    # 4. Fallback: coba parse sebagai literal Python (single quote, True/False, dll)
    try:
        obj = ast.literal_eval(candidate)
        return obj
    except Exception as e:
        # kalau masih gagal, lempar error dengan konteks yang jelas
        raise ValueError(
            f"Gagal parse output Agent 3 ke struktur data. "
            f"Detail error: {e}. Raw output (dipotong): {candidate[:500]}"
        )

In [19]:
# Cell 12.7 — Tool wrappers (mode hemat) [REVISI]

def product_search(query: str, top_k: int = 5):
    """
    Wrapper untuk search_product_tool.
    NOTE: pakai keyword arguments, jangan kirim dict tunggal.
    """
    return search_product_tool.run(
        query=query,
        top_k=top_k,
    )

def chemical_search(query: str, top_k: int = 5):
    """
    Wrapper untuk search_chemical_tool.
    """
    return search_chemical_tool.run(
        query=query,
        top_k=top_k,
    )

def innovation_check(compound_id1: str, compound_id2: str):
    """
    Wrapper untuk check_innovation_tool.
    """
    return check_innovation_tool.run(
        compound_id1=compound_id1,
        compound_id2=compound_id2,
    )


##**Run Pipeline / Kickoff**

In [20]:
# Cell 13 — PIPELINE MODE HEMAT (GENERAL, user bisa input apa saja)

def run_mode_hemat(user_request: str, max_chemicals: int = 2):
    """
    MODE HEMAT (GENERAL):

    - User cukup kirim teks bebas (mis: "buatin skincare buat kulit kering sensitif pakai AHA BHA").
    - Fungsi ini akan:
        1) Cari produk relevan dengan product_search (search_product_tool).
        2) Deteksi bahan aktif paling relevan dari user_request dengan chemical_search.
        3) Jika >= 2 bahan ditemukan, cek interaksi/inovasi dengan innovation_check.
        4) Bangun konteks dan panggil Gemini 2.5 Flash SATU KALI di akhir.

    Tidak pakai crew.kickoff, sangat hemat kuota.
    """

    print("=== MODE HEMAT (GENERAL) ===\n")

    # 1) Cari produk relevan berdasarkan request user
    print("[1] Cari produk relevan berdasarkan request user ...")
    produk = product_search(
        query=user_request,
        top_k=5,
    )

    # 2) Deteksi bahan kimia paling relevan dari user_request
    print("\n[2] Deteksi bahan aktif yang relevan dari user_request ...")
    chem_candidates = chemical_search(
        query=user_request,
        top_k=max_chemicals,
    )

    # Pastikan dalam bentuk list
    chem_list = []
    if isinstance(chem_candidates, list):
        chem_list = chem_candidates
    else:
        try:
            import json
            chem_list = json.loads(chem_candidates)
        except Exception:
            chem_list = []

    # Ambil label bahan yang manusiawi dari info / name_display
    detected_chemicals = []  # mis: ["Niacinamide", "Hyaluronic Acid"]
    for item in chem_list:
        if not isinstance(item, dict):
            continue
        info = item.get("info", "")
        name_display = item.get("name_display", "")
        human_name = None

        # Coba ambil dari info sebelum koma pertama: "Niacinamide, CxHyOz, ..."
        if isinstance(info, str) and "," in info:
            human_name = info.split(",")[0].strip()
        # Kalau nggak ada, fallback ke text dalam tanda kurung name_display: "CMPxxxx (Retinol)"
        if (not human_name) and isinstance(name_display, str) and "(" in name_display and ")" in name_display:
            human_name = name_display.split("(", 1)[-1].split(")", 1)[0].strip()

        # Fallback terakhir: pakai name_display apa adanya
        if not human_name and isinstance(name_display, str):
            human_name = name_display.strip()

        if human_name:
            detected_chemicals.append(human_name)

    # Buang duplikat sambil jaga urutan
    unique_chems = []
    for chem in detected_chemicals:
        if chem not in unique_chems:
            unique_chems.append(chem)

    # Ambil maksimal 2 bahan untuk cek interaksi
    chem_1_name = unique_chems[0] if len(unique_chems) >= 1 else None
    chem_2_name = unique_chems[1] if len(unique_chems) >= 2 else None

    print(f"   Bahan terdeteksi: {unique_chems}")

    chem_1_data = None
    chem_2_data = None
    inovasi = None

    # Kalau ada bahan 1, cari data detailnya lagi (boleh reuse chem_candidates, tapi ini eksplisit)
    if chem_1_name:
        print(f"\n[3] Ambil detail data bahan: {chem_1_name} ...")
        chem_1_data = chemical_search(
            query=chem_1_name,
            top_k=5,
        )

    if chem_2_name:
        print(f"\n[4] Ambil detail data bahan: {chem_2_name} ...")
        chem_2_data = chemical_search(
            query=chem_2_name,
            top_k=5,
        )

    # 3) Cek inovasi / interaksi jika ada 2 bahan
    if chem_1_name and chem_2_name:
        print(f"\n[5] Cek inovasi / interaksi {chem_1_name} + {chem_2_name} ...")
        inovasi = innovation_check(
            compound_id1=chem_1_name,
            compound_id2=chem_2_name,
        )

    # 4) Susun konteks menjadi satu string untuk 1x panggilan LLM
    print("\n[6] Susun konteks untuk LLM (mode hemat) ...")
    context_parts = []

    context_parts.append("=== DATA REQUEST USER ===")
    context_parts.append(user_request)

    context_parts.append("\n=== HASIL PENCARIAN PRODUK RELEVAN ===")
    context_parts.append(str(produk))

    if chem_1_name is not None:
        context_parts.append(f"\n=== DATA BAHAN KIMIA TERDETEKSI #1: {chem_1_name} ===")
        context_parts.append(str(chem_1_data))

    if chem_2_name is not None:
        context_parts.append(f"\n=== DATA BAHAN KIMIA TERDETEKSI #2: {chem_2_name} ===")
        context_parts.append(str(chem_2_data))

    if inovasi is not None:
        context_parts.append("\n=== HASIL CEK INOVASI / INTERAKSI ===")
        context_parts.append(str(inovasi))

    context_text = "\n".join(context_parts)

    # 5) Satu kali panggilan Gemini 2.5 Flash
    print("\n[7] Panggil Gemini 2.5 Flash SEKALI (mode hemat, general) ...\n")

    if chem_1_name and chem_2_name:
        combi_text = f"kombinasi {chem_1_name} dan {chem_2_name}"
    elif chem_1_name:
        combi_text = f"bahan {chem_1_name}"
    elif chem_2_name:
        combi_text = f"bahan {chem_2_name}"
    else:
        combi_text = "bahan-bahan aktif yang relevan dengan masalah user"

    prompt = f"""
Kamu adalah ahli formulasi skincare dan keamanan bahan.

User bertanya:
\"\"\"{user_request}\"\"\"


Berikut data yang sudah dikumpulkan (produk relevan, bahan aktif terdeteksi, dan hasil cek inovasi jika ada):
\"\"\"{context_text}\"\"\"


Tugasmu:

1. Jelaskan dengan bahasa awam apa yang penting dari {combi_text} (jika ada),
   termasuk:
   - efek utama di kulit,
   - boleh / aman atau tidak jika dikombinasikan,
   - pH ideal dan potensi iritasi,
   - jenis kulit yang cocok.

2. Jika user seperti meminta "buatin skincare" atau "rekomendasi routine/produk":
   - buat skincare routine yang relevan (pagi & malam jika perlu),
   - jelaskan secara singkat alasan pemilihan bahan & keamanannya.

3. Jika ada potensi risiko dari kombinasi bahan, jelaskan dengan jujur dan
   berikan alternatif yang lebih aman.

WAJIB:
- Jawab dalam Bahasa Indonesia.
- Fokus, ringkas, dan to the point (mode hemat token).
- Jangan gunakan markdown berlebihan; cukup teks biasa yang rapi.

Sekarang berikan jawabanmu.
"""

    final_answer = call_gemini_once(prompt)

    print("=== JAWABAN AKHIR (MODE HEMAT - GENERAL) ===\n")
    print(final_answer)

    return final_answer

In [21]:
user_request = "buatkan sunscreen untuk kulit sensitif"

hasil_mode_hemat = run_mode_hemat(user_request)

=== MODE HEMAT (GENERAL) ===

[1] Cari produk relevan berdasarkan request user ...
Using Tool: search_product_tool

[2] Deteksi bahan aktif yang relevan dari user_request ...
Using Tool: search_chemical_tool
   Bahan terdeteksi: ['Disodium EDTA']

[3] Ambil detail data bahan: Disodium EDTA ...
Using Tool: search_chemical_tool

[6] Susun konteks untuk LLM (mode hemat) ...

[7] Panggil Gemini 2.5 Flash SEKALI (mode hemat, general) ...

=== JAWABAN AKHIR (MODE HEMAT - GENERAL) ===

Halo! Sebagai ahli formulasi skincare, saya akan membantu Anda membuat rekomendasi sunscreen dan rutinitas untuk kulit sensitif.

---

1.  **Penjelasan Bahan: Disodium EDTA**

    *   **Catatan Penting:** Data yang Anda berikan di bawah judul "DATA BAHAN KIMIA TERDETEKSI #1: Disodium EDTA" sebenarnya berisi informasi tentang "Cocamidopropyl Betaine". Saya akan menjelaskan Disodium EDTA berdasarkan informasi yang benar, bukan dari data yang salah.
    *   **Efek Utama di Kulit:** Disodium EDTA adalah chelating a

In [22]:
%%writefile requirements.txt
# Core backend
fastapi==0.115.5
uvicorn[standard]==0.32.1
pydantic==2.10.2

# Agentic AI
crewai==0.152.0
crewai-tools==0.55.0
litellm==1.74.3

# Google Gemini SDK
google-genai==0.7.0

# Data & similarity
pandas==2.2.3
numpy==2.1.3
scikit-learn==1.5.2

# Optional (env)
python-dotenv==1.0.1




Writing requirements.txt
